In [ ]:
import numpy as np
import cupy as cp
import h5py
from holotomocupy.holo import G, GT
from holotomocupy.shift import S, ST
from holotomocupy.recon_methods import multiPaganin
from holotomocupy.utils import *
from holotomocupy.proc import remove_outliers
# Use managed memory
# cp.cuda.set_allocator(cp.cuda.MemoryPool(cp.cuda.malloc_managed).malloc)

# Init data sizes and parametes of the PXM of ID16A

In [ ]:
n = 2048  # object size in each dimension
detector_pixelsize = 3.03751e-6
energy = 33.35  # [keV] xray energy
wavelength = 1.2398419840550367e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
sx0 = 1.286e-3
z1 = np.array([4.236e-3,4.3625e-3,4.86850e-3,5.91950e-3])-sx0
z2 = focusToDetectorDistance-z1
distances = (z1*z2)/focusToDetectorDistance
magnifications = focusToDetectorDistance/z1
norm_magnifications = magnifications/magnifications[0]
voxelsize = np.abs(detector_pixelsize/magnifications[0]*2048/n)  # object voxel size

show = True
path = f'/data/vnikitin/ESRF/ID16A/20240924/AtomiumS2/'
pfile = f'AtomiumS2_HT_007nm'
path_out = f'/data/vnikitin/ESRF/ID16A/20240924_rec/AtomiumS2/'
print(f'{voxelsize=}')
ntheta=1800
ndist=4
st=0

## Read data

In [ ]:
data00 = np.zeros([ntheta,ndist,2048,2048],dtype='float32')
ref00 = np.zeros([ndist,2048,2048],dtype='float32')
ref01 = np.zeros([ndist,2048,2048],dtype='float32')
dark00 = np.zeros([ndist,2048,2048],dtype='float32')
    
mmeans = np.zeros(8)

for k in range(ndist):
    for j in range(0,ntheta):
        jtheta=st+j*1800//ntheta
        fname = f'{path}{pfile}_{k+1}_/{pfile}_{k+1}_{jtheta:04}.edf'
        # print(fname)
        tmp = dxchange.read_edf(fname)[0]
        data00[j,k] = tmp
    
    tmp = np.zeros([n,n],dtype='float32')
    for l in range(20):
        fname=f'{path}{pfile}_{k+1}_/ref{l:04}_0000.edf'
        #print(fname)
        tmp += dxchange.read_edf(fname)[0]
    tmp/=20
    ref00[k] = tmp

    tmp = np.zeros([n,n],dtype='float32')
    for l in range(20):
        fname = f'{path}{pfile}_{k+1}_/ref{l:04}_1800.edf'
        #print(fname)
        tmp += dxchange.read_edf(fname)[0]
    tmp/=20
    ref01[k] = tmp

    tmp = np.zeros([n,n],dtype='float32')
    for l in range(20):
        fname = f'{path}{pfile}_{k+1}_/darkend{l:04}.edf'
        #print(fname)
        tmp += dxchange.read_edf(fname)[0]
    tmp/=20

    dark00[k] = tmp



In [ ]:
data = data00.copy()
ref0 = ref00.copy()
ref1 = ref01.copy()
dark = dark00.copy()
# dark = np.mean(dark,axis=0)[np.newaxis]
# ref = np.mean(ref,axis=0)[np.newaxis]
data-=dark
ref0-=dark
ref1-=dark

data[data<0]=0
ref0[ref0<0]=0
ref1[ref1<0]=0

data[:,:,1320//3:1320//3+25//3,890//3:890//3+25//3] = data[:,:,1280//3:1280//3+25//3,890//3:890//3+25//3]
ref0[:,1320//3:1320//3+25//3,890//3:890//3+25//3] = ref0[:,1280//3:1280//3+25//3,890//3:890//3+25//3]
ref1[:,1320//3:1320//3+25//3,890//3:890//3+25//3] = ref1[:,1280//3:1280//3+25//3,890//3:890//3+25//3]

mshow(data[0,0],show)
for k in range(ndist):
    radius = 3
    threshold = 1.2
    for j in range(0,60):
        st=j*30
        end=st+30
        data[st:end,k] = remove_outliers(data[st:end,k], radius, threshold)
    
ref0[:] = remove_outliers(ref0[:], radius, threshold)     
ref1[:] = remove_outliers(ref1[:], radius, threshold)   

In [ ]:
print(data.shape)
shifts_random = np.zeros([ntheta,ndist,2],dtype='float32')
ss = np.zeros([ntheta,ndist,3])
for k in range(ndist):
    #s = np.loadtxt(f'{path}{pfile}_{k+1}_/correct.txt').astype('float32')[st:1800:1800//ntheta]/norm_magnifications[k]    
    ss[:,k] = np.loadtxt(f'{path}{pfile}_{k+1}_/attributes.txt').astype('float32')[::1800//ntheta]

mmean = np.mean(data[:,:,50:50+256,-50-256:-50],axis=(2,3))    
print(mmean.shape)
for k in range(4):
    plt.plot(mmean[:,k]/mmean[mmean.shape[0]//2,k],label='mean')
    plt.plot(ss[:,k]/ss[ss.shape[0]//2,k],label='current/counts')
    plt.legend()
    plt.show()


In [ ]:
mmean = np.mean(data[:,:,50:50+256,-50-256:-50],axis=(2,3))
mymean = mmean[mmean.shape[0]//2,1]
# mmean /= mymean
for k in range(4):
    plt.plot(mmean[:,k]/mymean,label='mean')
    plt.plot(ss[:,k]/ss[ss.shape[0]//2,k],label='current/counts')
    plt.legend()
    plt.show()

In [ ]:
mmeanref0 = np.mean(ref0[:,50:50+256,-50-256:-50],axis=(1,2))
mmeanref1 = np.mean(ref1[:,50:50+256,-50-256:-50],axis=(1,2))
# np.save('amps',mmean)
# np.save('amps_ref0',mmeanref0)
# np.save('amps_ref1',mmeanref1)
print(mmeanref0,mmeanref1)

